Import relevant Libaries 

In [ ]:
# Base libraries
import pandas as pd
import numpy as np
import os
import math as math

# Visualizations
import plotly.express as px
import plotly.graph_objects as go

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

GC Net Data

In [ ]:
gc = pd.read_parquet('/content/drive/MyDrive/Master_Thesis/data/df_hourly.gzip', engine='pyarrow')

*Test on one station*

In [ ]:
gc = gc[gc['station_name'] == "Humboldt"] 

NameError: ignored

In [ ]:
gc.info()

NameError: ignored

Promice Data

In [ ]:
pc = pd.read_parquet('/content/drive/MyDrive/Master_Thesis/data/promice_daily.gzip', engine='pyarrow')

In [ ]:
pc

*Test on one station*

In [ ]:
# define variables

# *Mandatory: Data
data = "Promice"

# *Mandatory: Date
date = "22-07-2008"

# Optional: Station
#station = "SCO_L_hour_v03"
station = "THU_L_day_v03"

# Optional: Measure
y = "All"

In [ ]:
# subset dataframe 
pc = pc[pc['file'] == station] 

In [ ]:
# select dato 
year= 2022
month = 1
day = 12

# subset df with date and find day of year and day variable
date_df = pc.loc[(pc['Datetime'].dt.year == year) & (pc['Datetime'].dt.month == month) & (pc['Datetime'].dt.day == day)]
  # select day of year
day =  date_df["DayOfYear"].mean()
datetime = date_df["Datetime"].max()

# select the specific day of century and the related values
day_century = date_df["DayOfCentury"].mean() 
day_century_value = pd.DataFrame(pd.DataFrame.mean(date_df)).reset_index()
day_century_value = pd.pivot_table(day_century_value, index=None, columns=['index'], aggfunc=max)

# group by calender day
pc_group = pc.loc[pc['DayOfYear'] == day]

<ipython-input-6-3a67d7402004>:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  day_century_value = pd.DataFrame(pd.DataFrame.mean(date_df)).reset_index()
<ipython-input-6-3a67d7402004>:14: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  day_century_value = pd.DataFrame(pd.DataFrame.mean(date_df)).reset_index()


In [ ]:
def get_percentile(df, date):
  """
  Function to return the percentile of specified values given a selected date.
  
  Parameters:
  df (pd.DataFrame): The dataframe to be used.
  date (str): The date of the observations for which the percentile is calculated.
  
  Returns:
  df (pd.DataFrame): A dataframe with all numerical columns and the percentiles of the values of the selected date.
  """

  # Find the index with the specified date
  row_index = int(df[df['Datetime'] == date].index[0])
  
  # Remove columns that do not contain numerical values & Subset df based on measure selection
  if y == "All":
    df = df.select_dtypes(include=['int', 'float']).copy()
  else:
    df = pd.DataFrame(df[y])
    df = df.select_dtypes(include=['int', 'float']).copy()
  
  # Create an empty dictionary for the output
  percentile_dict = {}
  
  # Iterate through the columns
  for col in df.columns:
    # Find the percentile of the value in the specified row and date
    value = df[col][row_index]
    if(math.isnan(value)): 
      percentile = math.nan
    else: 
      # calculate the percentile with df.rank()
      percentile = df[col].rank(pct=True)[df[col] == value].iloc[0]*100
    
    # Add the percentile to the dictionary
    #percentile_dict[col + "_percentile"] = percentile
    percentile_dict[col] = percentile
  
  
  # Create a dataframe with the output
  df_percentiles = pd.DataFrame(percentile_dict, index=[0])
  return df_percentiles

In [ ]:
x = get_percentile(pc, "2022-01-12")

In [ ]:
#x = x.append(day_century_value.iloc[0], ignore_index=True)

In [ ]:
x = get_percentile(pc_group, "2022-01-12")

In [ ]:
import prettytable as pt

table = pt.PrettyTable()
table.field_names = ["Measurement", "Percentile"]

for col in x.columns:
    if x[col].values > 90 or x[col].values < 10:
        table.add_row([f"\033[1m{col}\033[0m", f"\033[1m{int(x[col].values) if not np.isnan(x[col].values) else 'NaN'}% \033[0m"])
    else:
        table.add_row([col, f"{int(x[col].values) if not np.isnan(x[col].values) else 'NaN'}%"])


print(
f" Date: {date} \n" ,
f"Station: {station} \n" , 
f"Measurment: {y} \n"  , 
f"Data: {data} \n" ,
"-----------------------------------------------------------\n",
f"                   Climatology \n" 
)

print(table)

 Date: 22-07-2008 
 Station: THU_L_day_v03 
 Measurment: All 
 Data: Promice 
 -----------------------------------------------------------
                    Climatology 

+----------------------------------+------------+
|           Measurement            | Percentile |
+----------------------------------+------------+
|               Year               |    99%     |
|           MonthOfYear            |    4%      |
|            DayOfMonth            |    37%     |
|            DayOfYear             |    3%      |
|           DayOfCentury           |   100%     |
|         AirPressure(hPa)         |    3%      |
|        AirTemperature(C)         |    33%     |
|    AirTemperatureHygroClip(C)    |    34%     |
|       RelativeHumidity(%)        |    93%     |
|      SpecificHumidity(g/kg)      |    NaN%    |
|          WindSpeed(m/s)          |    7%      |
|         WindDirection(d)         |    48%     |
|      SensibleHeatFlux(W/m2)      |    NaN%    |
|       LatentHeatFlux(W/m2

*Visual test suite*

In [ ]:
fig = px.line(pc, x='Datetime', y="AirTemperature(C)")

fig.add_trace(go.Scatter(x=[datetime], y=[day_century_value], mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 15))

fig.show()

In [ ]:
ff = pc_group.select_dtypes(include=['int', 'float']).copy()
x = day_century_value

NameError: ignored

In [ ]:
ff = pd.DataFrame({'A': [2, 3, 4, 5, 6], 'B': [8, 7, 6, 5, 4], 'C': [4, 5, 6, 7, 8], 'D': [1, 4, 6, 7, 8], 'E': [2, 3, 5, 4, 7]})
x = pd.DataFrame({'A': [2], 'B': [7], 'C': [6]})

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


# Create a list of columns to be plotted
columns_to_plot = [col for col in x.columns.values if col in ff.columns.values]

# Create a list of values from x to be highlighted
values_to_highlight = x[columns_to_plot].values.flatten().tolist()


# Define the app
#app = dash.Dash()
app = JupyterDash(__name__)
# Set up the layout of the app
app.layout = html.Div([
    # Selection box
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'A', 'value': 'A'},
            {'label': 'B', 'value': 'B'},
            {'label': 'C', 'value': 'C'},
            {'label': 'D', 'value': 'D'},
            {'label': 'E', 'value': 'E'}
        ],
        value = 'A' # Set the initial value
    ),
    # Plotly graph
    dcc.Graph(
        id='graph',
        figure={
            'data': [{
                'x':ff[columns_to_plot],
                'y':ff[columns_to_plot],
                'type': 'box',
                'marker': {
                    'color': 'rgb(0, 0, 128)',
                    'opacity': 0.5
                }
            }],
            'layout': {
                'title': 'Boxplot for Climatology',
                'xaxis': {'title': ''},
                'yaxis': {'title': ''},
            }
        }
    )
])

# Define the callback to update the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('dropdown', 'value')])
def update_figure(selected_variable):
    # Create a list of columns to be plotted
    columns_to_plot = [col for col in x.columns.values if col in ff.columns.values]
    # Create a list of values from x to be highlighted
    values_to_highlight = x[columns_to_plot].values.flatten().tolist()
    fig = px.box(ff[columns_to_plot])
    fig.update_layout(title_text='Boxplot for Climatology', xaxis_title='' , yaxis_title='')
    # Highlight the values from x
    for c, v in zip(columns_to_plot, values_to_highlight):
      fig.add_scatter(x=[c], y=[v], name=f"{c} Selected Value",
                             mode = 'markers',
                             marker_symbol = 'circle-dot',
                             marker_size = 8,
                             marker_color = 'red')
    return fig


# Run app and display result inline in the notebook

#app.run_server(mode='inline')
app.run_server(mode='inline',host="0.0.0.0",port=1005)
# Run the app
#if __name__ == '__main__':
#    app.run_server(debug=True, mode = 'inline')

ConnectionError: ignored

In [ ]:
!pip install jupyter-dash
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd

# Load the data
#ff = pd.DataFrame({'A': [2, 3, 4, 5, 6], 'B': [8, 7, 6, 5, 4], 'C': [4, 5, 6, 7, 8], 'D': [1, 4, 6, 7, 8], 'E': [2, 3, 5, 4, 7]})
#x = pd.DataFrame({'A': [2], 'B': [7], 'C': [6]})

# Create the Dash app
app = dash.Dash()

# Create a list of columns to be plotted
columns_to_plot = [col for col in x.columns.values if col in ff.columns.values]

# Create a list of values from x to be highlighted
values_to_highlight = x[columns_to_plot].values.flatten().tolist()

# Create the boxplot
fig = px.box(ff[columns_to_plot])

# Format the axes and show the plot
fig.update_layout(title_text='Boxplot for Climatology', xaxis_title='' , yaxis_title='')

# Highlight the values from x
for c, v in zip(columns_to_plot, values_to_highlight):
  fig.add_scatter(x=[c], y=[v], name=f"{c} Selected Value",
                         mode = 'markers',
                         marker_symbol = 'circle-dot',
                         marker_size = 8,
                         marker_color = 'red')

# Add the interactive selection box
app.layout = html.Div([
    dcc.Dropdown(
        id='variable-selector',
        options=[{'label': i, 'value': i} for i in columns_to_plot],
        value='A'
    ),
    dcc.Graph(id='plot-boxplot')
])

@app.callback(
    dash.dependencies.Output('plot-boxplot', 'figure'),
    [dash.dependencies.Input('variable-selector', 'value')])
def update_figure(selected_variable):
    # Filter by selected variable
    fig = px.box(ff[selected_variable])

    # Format the axes and show the plot
    fig.update_layout(title_text='Boxplot for Climatology', xaxis_title='' , yaxis_title='')

    # Highlight the values from x
    for c, v in zip(columns_to_plot, values_to_highlight):
      fig.add_scatter(x=[c], y=[v], name=f"{c} Selected Value",
                             mode = 'markers',
                             marker_symbol = 'circle-dot',
                             marker_size = 8,
                             marker_color = 'red')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
import plotly.express as px

# Create a list of columns to be plotted
columns_to_plot = [col for col in x.columns.values if col in ff.columns.values]

# Create a list of values from x to be highlighted
values_to_highlight = x[columns_to_plot].values.flatten().tolist()

# Create the boxplot
fig = px.box(ff[columns_to_plot], orientation = "v",boxmode='group',points = )

# Format the axes and show the plot
fig.update_layout(title_text='Boxplot for Climatology', xaxis_title='' , yaxis_title='')

# Highlight the values from x
for c, v in zip(columns_to_plot, values_to_highlight):
  fig.add_scatter(x=[c], y=[v], name=f"{c} Selected Value",
                         mode = 'markers',
                         marker_symbol = 'circle-dot',
                         marker_size = 8,
                         marker_color = 'red')

fig.show()

In [ ]:
import plotly.express as px

# Create a list of columns to be plotted
columns_to_plot = [col for col in x.columns.values if col in ff.columns.values]

# Create a list of values from x to be highlighted
values_to_highlight = x[columns_to_plot].values.tolist()

# Create the boxplot
fig = px.box(ff[columns_to_plot])

# Format the axes and show the plot
fig.update_layout(title_text='Boxplot for Climatology', xaxis_title='' , yaxis_title='')

# Highlight the values from x
for c, v in zip(columns_to_plot, values_to_highlight):
  fig.add_scatter(x=[c], y=[v], name="Highlighted Values",
                         mode = 'markers',
                         marker_symbol = 'star',
                         marker_size = 15)
fig.show()

In [ ]:
# A function that calculates the percentiles of every column and their values

def percentile_df(df):
    for col in df.columns:
        df[f'{col}_pcta'] = df[col].rank(pct=True)

    return df

In [ ]:
percentile_df(pc_group)

<ipython-input-112-622f834c664c>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Year,MonthOfYear,DayOfMonth,HourOfDay(UTC),DayOfYear,DayOfCentury,AirPressure(hPa),AirTemperature(C),AirTemperatureHygroClip(C),RelativeHumidity(%),...,ElevationGPS(m)_pcta,HorDilOfPrecGPS_pcta,LoggerTemperature(C)_pcta,FanCurrent(mA)_pcta,BatteryVoltage(V)_pcta,file_pcta,Month_pcta,Day_pcta,Hour_pcta,Datetime_pcta
0,2008,7,22,0,204,3126,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.144345,0.022321,0.002976
1,2008,7,22,1,204,3126,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.144345,0.063988,0.005952
2,2008,7,22,2,204,3126,946.18,3.03,2.61,78.5,...,NaN,NaN,0.354839,0.003226,0.025806,0.501488,0.501488,0.144345,0.105655,0.008929
3,2008,7,22,3,204,3126,946.44,2.97,2.43,81.5,...,0.021097,0.658228,0.141935,0.041935,0.058065,0.501488,0.501488,0.144345,0.147321,0.011905
4,2008,7,22,4,204,3126,946.90,2.79,2.34,82.4,...,0.900844,0.424051,0.151613,0.066129,0.050000,0.501488,0.501488,0.144345,0.188988,0.014881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,2021,7,23,19,204,7875,954.00,3.39,3.57,93.3,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.644345,0.813988,0.988095
113996,2021,7,23,20,204,7875,954.00,3.10,3.11,97.0,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.644345,0.855655,0.991071
113997,2021,7,23,21,204,7875,954.00,3.06,3.05,99.5,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.644345,0.897321,0.994048
113998,2021,7,23,22,204,7875,955.00,2.56,2.55,99.6,...,NaN,NaN,NaN,NaN,NaN,0.501488,0.501488,0.644345,0.938988,0.997024
